In [1]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.1-py3-none-any.whl size=16265 sha256=d10ac389965b6f871b671283fda8d9526334a9a31c3bc183a53ad05d1686498d
  Stored in directory: /root/.cache/pip/wheels/e8/81/8f/29aafd7d671a4e9db4e976d71728870173f0b3f48ccba0bc32
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    try:
      coverpage= soup.find_all('div',class_='entry-header')
      title=coverpage[0].find('h1').get_text()
      time = coverpage[0].find('div',class_='jeg_meta_date').get_text()
      # print(title)
    except:
      title=''
      time=''

    try:
      coverpage = soup.find_all('div', class_='content-inner ')
      article=''
      for i in coverpage[0].find_all('p'):
        article=article+i.get_text()
      # print(article)
    except:
      article=''
      print('error\n'+url)
  except:
    title=''
    time=''
    article=''
    print('url error\n'+url)    

  return [title,time,article]

In [6]:
get_articles('https://www.eldiario.net/portal/2022/07/19/mesa-cuestiona-corrupcion-e-ineptitud-en-la-procuraduria/')

['Mesa cuestiona corrupción e  ineptitud en la Procuraduría',
 '\n19/07/2022 5:58 am\n',
 'El expresidente, Carlos Mesa Gisbert, se refirió al laudo arbitral en el que se emplaza al país a cumplir con una indemnización mucho mayor al monto inicial demandado, señalando que se trata de un caso es similar al de Quiborax, donde el Estado debe pagar por la ineptitud y la corrupción de funcionarios de la Procuraduría, que no cumplen su función de defender al Estado.Desde las redes sociales, el exmandatario y líder de Comunidad Ciudadana condeno la forma en que autoridades del Movimiento al Socialismo han generado pérdidas “inaceptables” para el país.Mesa además adelantó un nuevo encubrimiento a los errores de funcionarios encargados de la defensa del Estado, como ya ocurrió durante el gobierno de Evo Morales.“Como ocurrió con Quiborax, el gobierno termina pagando más de lo que pedían los arbitrajes.\xa0 Por la ineptitud o la corrupción de los Procuradores masistas, se generan pérdidas inacep

In [7]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

df=pd.DataFrame()
read_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/eldiario.lapaz/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
output=df['rows'].parallel_apply(get_articles)

df['country']='Argentina'
df['title']=''
df['time']=''
df['article']=''
for i in range(0,len(df)):
  df.loc[i]['title','time','article']=output[i]


df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
name_article='/content/drive/MyDrive/Colab Notebooks/eldiario.lapaz/url_article.csv'
df.to_csv(name_article, index=False)
print(df)


INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


url error
error
                                                   page  \
0     https://www.eldiario.net/portal/category/polit...   
1     https://www.eldiario.net/portal/category/polit...   
2     https://www.eldiario.net/portal/category/polit...   
3     https://www.eldiario.net/portal/category/polit...   
4     https://www.eldiario.net/portal/category/polit...   
...                                                 ...   
5299  https://www.eldiario.net/portal/category/inter...   
5300  https://www.eldiario.net/portal/category/inter...   
5301  https://www.eldiario.net/portal/category/inter...   
5302  https://www.eldiario.net/portal/category/inter...   
5303  https://www.eldiario.net/portal/category/inter...   

                                                   rows    country  \
0     https://www.eldiario.net/portal/2022/07/19/com...  Argentina   
1     https://www.eldiario.net/portal/2022/07/19/exi...  Argentina   
2     https://www.eldiario.net/portal/2022/07/19/afi...  Argentin